In [ ]:
!pip install settrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95.6 MB 37 kB/s 
  Created wheel for settrade: filename=settrade-0.5.0-py3-none-any.whl size=5058070 sha256=dd2542fd4966e0c65aacb6c8585db872b77da5519938e9bed9a178379e7d5d81
  Stored in directory: /root/.cache/pip/wheels/1d/c9/b8/b82bf42b1de56957a7916cecd1416a65350900cc3967401177
Successfully built settrade


In [ ]:
#import libaries
import requests
import settrade.openapi
from settrade.openapi import Investor
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date


In [ ]:
#generate the dataframe of date and news, accordingly
def gen_news_df(num_page: int):

  # list of news and date 
  news_lst = list()
  date_lst = list()

  # loop from page #1 to #num_page
  for page in range(1,num_page+1):

    # retive html from website 
    raw_query = requests.get('https://daily.kaohoon.com/listed-companies/page/'+str(page))

    # get raw html from queried
    raw_html = BeautifulSoup(raw_query.text, 'lxml')

    # extract the header of thumnails
    for h2 in raw_html.find_all('h2', {'class': 'post-title'}):
      
      # append the extracted news to news_lst
      news_lst.append(h2.string)

    #extract the date from each news
    for date in raw_html.find_all('span', {'class': 'date meta-item tie-icon'}):

      # append the extracted date to date_lst
      date_lst.append(date.string)
    print('Page',page,'queried')
    
  # create a dataframe of date and news
  news_df = pd.DataFrame(list(zip(date_lst,news_lst)), columns = ['date', 'news'])

  return news_df

# filter non related news out of the string
def filter_non_related(data):
  
  # split data and check the index 0 with SET company list 
  if not data.split()[0] in companies_lst: 
    return 'non-related'

  # return in the old pattern 
  else:
    return ''.join(data)

# append a company name list of the news 
def get_only_name(data,name_lst):
  name_lst.append(data.split()[0])

# get the company name list 
def gen_company_name(df):
  name = list()
  df['news'].apply(get_only_name, name_lst = name)
  return name

# get the prices of stocks with the company name as key and datetime, price dataframe as items
def get_all_prices(news_stock_lst):
  
  # define the dicts.
  stock_dict = dict()
  
  # define the today's date.
  today = str(date.today()) + 'T00:00:00'

  # SET market object.
  market = investor.MarketData()
  
  # loop though all companies list.
  for stock in news_stock_lst:
    
    # query the price via the SETopenAPI for 1 day period limit for only 500.
    chart_data_res = market.get_candlestick(stock, "1d", 500, "2022-06-01T00:00:00", today, True) 
    
    if chart_data_res['success']:
      # create a dataframe of datetime and closed price of stock.
      df = pd.DataFrame(list(zip(chart_data_res['data']['time'],chart_data_res['data']['close'])), columns = ['datetime','price'])
      
      # remove the time from datetime 
      df['datetime'] = pd.to_datetime(df['datetime'], unit = 's').dt.date

      # add the dataframe to the stock_dict with a key as stock's name.
      stock_dict[stock] = df

    # print the queried stock 
    print('Queried',stock)
     
  return stock_dict  



In [ ]:
news_df = gen_news_df(num_page = 50)

Page 1 queried
Page 2 queried
Page 3 queried
Page 4 queried
Page 5 queried
Page 6 queried
Page 7 queried
Page 8 queried
Page 9 queried
Page 10 queried
Page 11 queried
Page 12 queried
Page 13 queried
Page 14 queried
Page 15 queried
Page 16 queried
Page 17 queried
Page 18 queried
Page 19 queried
Page 20 queried
Page 21 queried
Page 22 queried
Page 23 queried
Page 24 queried
Page 25 queried
Page 26 queried
Page 27 queried
Page 28 queried
Page 29 queried
Page 30 queried
Page 31 queried
Page 32 queried
Page 33 queried
Page 34 queried
Page 35 queried
Page 36 queried
Page 37 queried
Page 38 queried
Page 39 queried
Page 40 queried
Page 41 queried
Page 42 queried
Page 43 queried
Page 44 queried
Page 45 queried
Page 46 queried
Page 47 queried
Page 48 queried
Page 49 queried
Page 50 queried


In [ ]:
# read SET company from the excel file.
set_companies = pd.read_excel('set_companies.xlsx')
set_companies.head(3)

,Symbol,Company,Market,Industry,Sector,Address,Zip code,Tel.,Fax,Website
0,2S,2S METAL PUBLIC COMPANY LIMITED,SET,Industrials,Steel and Metal Products,"8/5 Moo 14, Tha-change, Bangklam District Song...",90110,0-7480-0111,0-7480-1286,www.ss.co.th
1,3K-BAT,THAI ENERGY STORAGE TECHNOLOGY PUBLIC COMPANY ...,SET,Industrials,Automotive,"387 Moo 4, Sukhumvit Rd., Phraek Sa Sub-distri...",10280,0-2709-3535,0-2393-2509 / 0-2709-4965,-
2,7UP,SEVEN UTILITIES AND POWER PUBLIC COMPANY LIMITED,SET,Resources,Energy & Utilities,"73 Mahachol Building, Soi Sukhumvit 62, Sukhum...",10260,0-2741-5700,0-2741-5724,www.sevenup.co.th


In [ ]:
# convert stock symbols to list
companies_lst = set_companies['Symbol'].to_list()
print(companies_lst)

['2S', '3K-BAT', '7UP', 'A', 'A5', 'AAV', 'ABICO', 'ABM', 'ABPIF', 'ACAP', 'ACC', 'ACE', 'ACG', 'ADB', 'ADD', 'ADVANC', 'AEONTS', 'AF', 'AFC', 'AGE', 'AH', 'AHC', 'AI', 'AIE', 'AIMCG', 'AIMIRT', 'AIRA', 'AIT', 'AJ', 'AJA', 'AKP', 'AKR', 'ALL', 'ALLA', 'ALLY', 'ALPHAX', 'ALT', 'ALUCON', 'AMA', 'AMANAH', 'AMARIN', 'AMATA', 'AMATAR', 'AMATAV', 'AMC', 'AMR', 'ANAN', 'AOT', 'AP', 'APCO', 'APCS', 'APEX', 'APP', 'APURE', 'AQ', 'AQUA', 'ARIN', 'ARIP', 'ARROW', 'AS', 'ASAP', 'ASEFA', 'ASIA', 'ASIAN', 'ASIMAR', 'ASK', 'ASN', 'ASP', 'ASW', 'ATP30', 'AU', 'AUCT', 'AWC', 'AYUD', 'B', 'B52', 'BA', 'BAFS', 'BAM', 'BANPU', 'BAY', 'BBGI', 'BBIK', 'BBL', 'BC', 'BCH', 'BCP', 'BCPG', 'BCT', 'BDMS', 'BE8', 'BEAUTY', 'BEC', 'BEM', 'BEYOND', 'BFIT', 'BGC', 'BGRIM', 'BGT', 'BH', 'BIG', 'BIOTEC', 'BIS', 'BIZ', 'BJC', 'BJCHI', 'BKD', 'BKI', 'BKKCP', 'BLA', 'BLAND', 'BLESS', 'BLISS', 'BM', 'BOFFICE', 'BOL', 'BPP', 'BR', 'BRI', 'BROCK', 'BROOK', 'BRR', 'BRRGIF', 'BSBM', 'BSM', 'BTNC', 'BTS', 'BTSGIF', 'BTW', 'BUI

In [ ]:
# filter non-related news
news_df['news'] = news_df['news'].apply(filter_non_related)

# remove the 'non-related' and reset indexes 
news_df = news_df[news_df['news'] != 'non-related'].reset_index()

#drop the old index
news_df = news_df.drop(['index'], axis = 1)

news_df

,date,news
0,12/09/2022,JKN ปั้นสถานีข่าวเศรษฐกิจ 24 ชม. ผนึก AIS PLAY...
1,12/09/2022,NSL การันตียอดขายปีนี้ 3.9 พันล้าน เตรียมเปิดต...
2,12/09/2022,CPF ได้มาตรฐาน LR Farm F1rst ยกระดับผลิตเนื้อไ...
3,12/09/2022,PIN เร่งสร้างรายได้ประจำ 3 ปีดันสัดส่วนเพิ่มเป...
4,12/09/2022,PTT ห่วงราคา LNG พุ่งยืนเหนือ 50 ดอลล์ แนะลดปร...
...,...,...
380,21/07/2022,BTS ซื้อเป้า 12.30 บาท ลุ้น Q1 มีกำไร 869 ล้าน...
381,20/07/2022,THG ส่งซิกไตรมาส 3 เข้าไฮซีซั่น ชี้มาตรการแบงก...
382,20/07/2022,PREB เปิดแนวราบ 2 โครงการ Q3 เล็งซื้อที่ดินย่า...
383,20/07/2022,MJD จับเมกะเทรนด์โลก เปิดโครงการแฟล็กชิพ มอลตั...


In [ ]:
#get all company names from news_df 
name = gen_company_name(news_df)

# create new columns in df
news_df['comp_name'] = name

# rearrange columns in df
news_df = news_df[['date','comp_name','news']]

# convert object to datetime type
news_df['date'] = pd.to_datetime(news_df['date'], format = '%d/%m/%Y')

# remove time for datetime
news_df['date'] = news_df['date'].dt.date

news_df

,date,comp_name,news
0,2022-09-12,JKN,JKN ปั้นสถานีข่าวเศรษฐกิจ 24 ชม. ผนึก AIS PLAY...
1,2022-09-12,NSL,NSL การันตียอดขายปีนี้ 3.9 พันล้าน เตรียมเปิดต...
2,2022-09-12,CPF,CPF ได้มาตรฐาน LR Farm F1rst ยกระดับผลิตเนื้อไ...
3,2022-09-12,PIN,PIN เร่งสร้างรายได้ประจำ 3 ปีดันสัดส่วนเพิ่มเป...
4,2022-09-12,PTT,PTT ห่วงราคา LNG พุ่งยืนเหนือ 50 ดอลล์ แนะลดปร...
...,...,...,...
380,2022-07-21,BTS,BTS ซื้อเป้า 12.30 บาท ลุ้น Q1 มีกำไร 869 ล้าน...
381,2022-07-20,THG,THG ส่งซิกไตรมาส 3 เข้าไฮซีซั่น ชี้มาตรการแบงก...
382,2022-07-20,PREB,PREB เปิดแนวราบ 2 โครงการ Q3 เล็งซื้อที่ดินย่า...
383,2022-07-20,MJD,MJD จับเมกะเทรนด์โลก เปิดโครงการแฟล็กชิพ มอลตั...


In [ ]:
# Get all stock name as list 
news_stock_lst = news_df['comp_name'].unique()

# get price history of all stocks 
prices = get_all_prices(news_stock_lst)

Queried JKN
Queried NSL
Queried CPF
Queried PIN
Queried PTT
Queried MC
Queried TRU
Queried WINMED
Queried OR
Queried AMR
Queried SNNP
Queried D
Queried ONEE
Queried UPOIC
Queried XO
Queried BA
Queried ACG
Queried KSL
Queried AWC
Queried LST
Queried NER
Queried EKH
Queried WPH
Queried RS
Queried TGPRO
Queried UBE
Queried LALIN
Queried SISB
Queried SIRI
Queried SPALI
Queried III
Queried LEO
Queried SC
Queried MAJOR
Queried ZEN
Queried PRM
Queried WP
Queried CAZ
Queried MINT
Queried CWT
Queried GUNKUL
Queried AMATA
Queried ASW
Queried SSP
Queried SABUY
Queried LHK
Queried UAC
Queried ANAN
Queried FPT
Queried CKP
Queried PORT
Queried DTAC
Queried AMATAV
Queried BRI
Queried TMILL
Queried DDD
Queried SELIC
Queried SENAJ
Queried JDF
Queried PRINC
Queried CMO
Queried TEAMG
Queried BBIK
Queried VGI
Queried RT
Queried SCGP
Queried MAKRO
Queried SAPPE
Queried QH
Queried LPN
Queried AIT
Queried SGP
Queried SPCG
Queried KISS
Queried MEGA
Queried NDR
Queried RJH
Queried GPI
Queried SUN
Queried BGC
Q

In [ ]:
# define list for price 
prices_list = list()

# loop though each row in news_df 
for index,row in news_df.iterrows():
  
  # get the price df from the company name 
  stock_df = prices[row['comp_name']]

  # find the price of stock at the specific datetime 
  price_at_news = stock_df[stock_df['datetime'] == row['date']]

  # check if the price is not avaliable (equals to []).
  if price_at_news['price'].to_list() == []:
    
    # check if the lasted price is not avaliable (equals to []).
    if stock_df.tail(1)['price'].to_list() == []:

      # append the - as not avaliable resources. 
      prices_list.append('-') 

      # has the lasted updated price, but not avaliable at the specific datetime. 
    else:

      # extend the lasted updated price, instead of the specific datetime. 
      prices_list.extend(stock_df.tail(1)['price'].to_list())

  # price is avaliable at the specific date time
  else:
    
    #extend the prices_list with the price at the specific datetime
    prices_list.extend(price_at_news['price'].to_list())


In [ ]:
#create a dataframe in order to concat.
price_at_time_df = pd.DataFrame(prices_list, columns = ['price'])

# concat the news_df with the price_df.
news_df = pd.concat([news_df,price_at_time_df], axis = 1)

In [ ]:
news_df

,date,comp_name,news,price
0,2022-09-12,JKN,JKN ปั้นสถานีข่าวเศรษฐกิจ 24 ชม. ผนึก AIS PLAY...,4.26
1,2022-09-12,NSL,NSL การันตียอดขายปีนี้ 3.9 พันล้าน เตรียมเปิดต...,22.0
2,2022-09-12,CPF,CPF ได้มาตรฐาน LR Farm F1rst ยกระดับผลิตเนื้อไ...,25.75
3,2022-09-12,PIN,PIN เร่งสร้างรายได้ประจำ 3 ปีดันสัดส่วนเพิ่มเป...,3.62
4,2022-09-12,PTT,PTT ห่วงราคา LNG พุ่งยืนเหนือ 50 ดอลล์ แนะลดปร...,37.25
...,...,...,...,...
380,2022-07-21,BTS,BTS ซื้อเป้า 12.30 บาท ลุ้น Q1 มีกำไร 869 ล้าน...,13.9
381,2022-07-20,THG,THG ส่งซิกไตรมาส 3 เข้าไฮซีซั่น ชี้มาตรการแบงก...,24.0
382,2022-07-20,PREB,PREB เปิดแนวราบ 2 โครงการ Q3 เล็งซื้อที่ดินย่า...,8.7
383,2022-07-20,MJD,MJD จับเมกะเทรนด์โลก เปิดโครงการแฟล็กชิพ มอลตั...,3.08


In [ ]:
#TO_DO
# Detect the new high for 48 weeks in the week time frame.
# lib: Talib

In [ ]:
all_stocks_price = get_all_prices(companies_lst)

Queried 2S
Queried 3K-BAT
Queried 7UP
Queried A
Queried A5
Queried AAV
Queried ABICO
Queried ABM
Queried ABPIF
Queried ACAP
Queried ACC
Queried ACE
Queried ACG
Queried ADB
Queried ADD
Queried ADVANC
Queried AEONTS
Queried AF
Queried AFC
Queried AGE
Queried AH
Queried AHC
Queried AI
Queried AIE
Queried AIMCG
Queried AIMIRT
Queried AIRA
Queried AIT
Queried AJ
Queried AJA
Queried AKP
Queried AKR
Queried ALL
Queried ALLA
Queried ALLY
Queried ALPHAX
Queried ALT
Queried ALUCON
Queried AMA
Queried AMANAH
Queried AMARIN
Queried AMATA
Queried AMATAR
Queried AMATAV
Queried AMC
Queried AMR
Queried ANAN
Queried AOT
Queried AP
Queried APCO
Queried APCS
Queried APEX
Queried APP
Queried APURE
Queried AQ
Queried AQUA
Queried ARIN
Queried ARIP
Queried ARROW
Queried AS
Queried ASAP
Queried ASEFA
Queried ASIA
Queried ASIAN
Queried ASIMAR
Queried ASK
Queried ASN
Queried ASP
Queried ASW
Queried ATP30
Queried AU
Queried AUCT
Queried AWC
Queried AYUD
Queried B
Queried B52
Queried BA
Queried BAFS
Queried BAM


In [ ]:
all_stocks_price

{'2S':       datetime     price
 0   2022-05-31  4.240000
 1   2022-06-01  1.636380
 2   2022-06-05  1.636380
 3   2022-06-06  4.320000
 4   2022-06-07  1.628198
 ..         ...       ...
 65  2022-09-06  1.849109
 66  2022-09-07  3.440000
 67  2022-09-08  1.881837
 68  2022-09-11  1.881837
 69  2022-09-12  3.460000
 
 [70 rows x 2 columns], '3K-BAT':       datetime   price
 0   2022-06-05   80.00
 1   2022-06-06   81.75
 2   2022-06-07   73.25
 3   2022-06-08   81.75
 4   2022-06-09   81.50
 5   2022-06-12   73.25
 6   2022-06-13   92.50
 7   2022-06-14   88.50
 8   2022-06-19   70.50
 9   2022-06-21   95.00
 10  2022-06-23   95.00
 11  2022-06-26   74.50
 12  2022-06-28   88.00
 13  2022-07-04   97.75
 14  2022-07-05   75.50
 15  2022-07-06   89.00
 16  2022-07-13   93.00
 17  2022-07-14   73.25
 18  2022-07-19   93.00
 19  2022-07-20   95.50
 20  2022-07-24   70.25
 21  2022-07-25  118.50
 22  2022-08-01   98.00
 23  2022-08-03   71.75
 24  2022-08-04  100.00
 25  2022-08-08   59.25